## Imports

In [12]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [13]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import math
from sklearn.decomposition import PCA

In [14]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [15]:
path = Path('./data')

In [16]:
train_trans = pd.read_csv(path/'train_transaction.csv')
train_ids = pd.read_csv(path/'train_identity.csv')
test_trans = pd.read_csv(path/'test_transaction.csv')
test_ids = pd.read_csv(path/'test_identity.csv')

## Data Preparation

In [17]:
train_data = train_trans #pd.merge(train_trans, train_ids, on='TransactionID')
test_data = test_trans #pd.merge(test_trans, test_ids, on='TransactionID')

In [7]:
train_data['Date'] = train_data.TransactionDT.map(lambda x: int(x/86400))
test_data['Date'] = test_data.TransactionDT.map(lambda x: int(x/86400))

In [8]:
train_data['Hour'] = train_data.TransactionDT.map(lambda x: int((x%86400)/3600))
test_data['Hour'] = test_data.TransactionDT.map(lambda x: int((x%86400)/3600))

In [9]:
train_data.isnull()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,Date,Hour
110776,3405057,0,10555182,100.000,R,4425,562.0,150.0,mastercard,197.0,credit,441.0,87.0,NaN,NaN,aol.com,charter.net,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,143694.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,100.000000,NotFound,64.0,-300.0,New,NotFound,166.0,13.0,529.0,597.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Mac OS X 10_12_6,safari generic,24.0,1280x800,match_status:2,T,F,F,F,desktop,MacOS,122,3
42166

In [10]:
train_data.R_emaildomain.isnull().sum()/len(train_data)

0.09117192320758773

In [11]:
train_data.DeviceType.isnull().sum()/len(train_data)

0.023732432938370553

In [12]:
train_null_counts = train_data.isnull().sum()
train_null_counts.name = 'train_counts'
test_null_counts = test_data.isnull().sum()
test_null_counts.name = 'test_counts'

In [13]:
test_null_counts.sort_values(ascending=False)/len(train_data)

M3                0.983873
V2                0.983873
V7                0.983873
V10               0.983873
V6                0.983873
V5                0.983873
V11               0.983873
M2                0.983873
D11               0.983873
V4                0.983873
V9                0.983873
V3                0.983873
M1                0.983873
dist1             0.983873
V1                0.983873
M9                0.983873
M8                0.983873
M7                0.983873
M6                0.983873
M5                0.983873
V8                0.983873
id_24             0.951010
id_25             0.948937
id_26             0.948881
id_08             0.948798
id_07             0.948798
id_21             0.948798
id_23             0.948777
id_22             0.948777
id_27             0.948777
D2                0.740843
dist2             0.731310
D3                0.643785
id_18             0.631145
D5                0.597103
D7                0.579590
D12               0.528866
M

In [14]:
pd.concat([train_null_counts/len(train_data), test_null_counts/len(test_data)], axis=1).sort_values('train_counts', ascending=False)

/Users/vinodreddy/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


,train_counts,test_counts
V9,1.000000,1.000000
V1,1.000000,1.000000
M3,1.000000,1.000000
M5,1.000000,1.000000
M6,1.000000,1.000000
M7,1.000000,1.000000
M9,1.000000,1.000000
dist1,1.000000,1.000000
V10,1.000000,1.000000
M1,1.000000,1.000000


In [9]:
base_features = ['TransactionID' ,'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'Hour', 'P_emaildomain', 'R_emaildomain'] + list(train_data.columns)[17:31] 

In [10]:
train_data[base_features+['isFraud']].to_csv(path/'base_features_train.csv', index=False, header=True)
test_data[base_features].to_csv(path/'base_features_test.csv', index=False, header=True)